In [ ]:
import pandas as pd
import re
import os
import glob
import time
import selenium
import pytesseract
import chromedriver_autoinstaller
import tkinter
from tkinter import messagebox
import time
import keyboard

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ChromeOptions
from msedge.selenium_tools import Edge, EdgeOptions
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.opera import OperaDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

import pyautogui as py
from zipfile import ZipFile
import datetime
from datetime import timedelta
from datetime import date
import base64


from PIL import Image
from io import BytesIO
import io


#pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

import pytesseract

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df1=pd.read_excel(r"C:\Users\swast\Downloads\Person Epic ID.xlsx")

In [ ]:
driver = webdriver.Chrome(r"C:\Users\swast\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
driver.get('https://electoralsearch.eci.gov.in/')

In [ ]:
columns = ["S. NO.", "Epic Number", "Name", "Age", "Relative Name", "State", "District", "Assembly Constituency", "Part", "Polling Station", "Part Serial Number", "Action"]
df = pd.DataFrame(data, columns=columns)

In [ ]:
driver.find_element(By.XPATH,'//*[@id="epic"]').click()
time.sleep(2)
for i in df1['Person Voter Id'].unique():
    driver.find_element(By.XPATH,'//*[@id="epicID"]').send_keys(i)
    time.sleep(1)
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/div/div[3]/div[2]/div[2]/div[1]/div[2]/div[2]/div/select').send_keys('Madhya Pradesh')
    time.sleep(1)

    #Captcha Reading
    image_element = driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/div/div[3]/div[3]/div[1]/div/div[1]/div/img').get_attribute("src").split(",")[-1]
    image_bytes = io.BytesIO(base64.b64decode(image_element))
    image = Image.open(image_bytes)
    sharpened_image = image.filter(ImageFilter.SHARPEN)
    cv_image = np.array(sharpened_image)
    gray_image = cv2.cvtColor(cv_image, cv2.COLOR_RGB2GRAY)
    gray_image_float = gray_image.astype(np.float32) / 255.0
    denoised_image = np.uint8(gray_image_float * 255.0)
    preprocessed_image = Image.fromarray(denoised_image)
    text = pytesseract.image_to_string(preprocessed_image).replace('\n','')

    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/div/div[3]/div[3]/div[1]/div/div[2]/div[2]/div/input').send_keys(text)
    time.sleep(2)
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/div/div[3]/div[3]/div[2]/button[1]').click()

    while True:
        try:
            driver.find_element(By.XPATH,'/html/body/div/div[2]/div/div[4]/table')
            break
        except:
            driver.refresh()
            #print('Captcha Error')
            time.sleep(10)

            driver.find_element(By.XPATH,'//*[@id="epic"]').click()
            time.sleep(1)
            driver.find_element(By.XPATH,'//*[@id="epicID"]').send_keys(i)
            time.sleep(1)
            driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/div/div[3]/div[2]/div[2]/div[1]/div[2]/div[2]/div/select').send_keys('Madhya Pradesh')
            time.sleep(1)

            image_element = driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/div/div[3]/div[3]/div[1]/div/div[1]/div/img').get_attribute("src").split(",")[-1]
            image_bytes = io.BytesIO(base64.b64decode(image_element))
            image = Image.open(image_bytes)
            sharpened_image = image.filter(ImageFilter.SHARPEN)
            cv_image = np.array(sharpened_image)
            gray_image = cv2.cvtColor(cv_image, cv2.COLOR_RGB2GRAY)
            gray_image_float = gray_image.astype(np.float32) / 255.0
            denoised_image = np.uint8(gray_image_float * 255.0)
            preprocessed_image = Image.fromarray(denoised_image)
            text = pytesseract.image_to_string(preprocessed_image).replace('\n','')
            driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/div/div[3]/div[3]/div[1]/div/div[2]/div[2]/div/input').send_keys(text)
            time.sleep(2)
            driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/div/div[3]/div[3]/div[2]/button[1]').click()
            time.sleep(2)

    table = driver.find_element_by_id("table-id")
    rows = table.find_elements_by_tag_name("tr")
    data = []

    # Iterate through rows and extract data
    for row in rows[1:]:  # Skip the header row
        cells = row.find_elements_by_tag_name("td")
        row_data = [cell.text for cell in cells]
        data.append(row_data)

    for j in data:
        df.loc[len(df)]=j
    df.drop_duplicates(inplace=True)
    driver.find_element(By.XPATH,'//*[@id="epicID"]').clear()
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/div/div[3]/div[3]/div[1]/div/div[2]/div[2]/div/input').clear()
